In [104]:
import hazm
import parsivar
from string import punctuation
import json
import re
import numpy as np
from pathlib import Path
from collections import Counter
import collections
from hazm import Normalizer, word_tokenize, Stemmer
from collections import Counter
import json
import re
import string
from collections import Counter
from hazm import Normalizer, word_tokenize, Stemmer

In [105]:
def load_docs():
    docs = {}
    contents = []
    urls = []
    with open("IR_data_news_12k.json", 'r') as file:
        docs = json.load(file)
        for key in docs.keys():
            # index of files
            idx = str(key)
            # extract and save url, title and content of each doc
            docs[idx] = {'title': docs[idx]['title'],
                         'content': docs[idx]['content'],
                         'url': docs[idx]['url'],
                         }

            contents.append(docs[idx]['content'])
    return docs, contents, urls

In [106]:
docs, contents, urls = load_docs()
docs['6']

{'title': 'اعلام برنامه نشست خبری گل محمدی/ مجیدی هم باید به محل تمرین پرسپولیس برود!',
 'content': '\nبه گزارش خبرگزاری فارس و به نقل از\xa0 سایت باشگاه پرسپولیس، نشست خبری پیش از مسابقه سرمربیان دو تیم پرسپولیس و استقلال از هفته بیست و سوم لیگ برتر (جام خلیج فارس) با\xa0 مدیریت سازمان لیگ و هماهنگی\xa0باشگاه میزبان (پرسپولیس) در ورزشگاه شهید کاظمی به شرح زیر\xa0 برگزار می\u200cشود: چهارشنبه ۲۵ اسفند ساعت ۱۳ فرهاد مجیدی سرمربی استقلال ساعت ۱۳:۳۰ یحیی گل محمدی سرمربی پرسپولیس \xa0مسابقه دو تیم روز پنجشنبه در ورزشگاه آزادی برگزار می\u200cشود. \xa0به گزارش خبرگزاری فارس، پیش از این باشگاه استقلال اعلام کرده بود قرار است نشست خبری فرهاد مجیدی، سرمربی این تیم از ساعت ۱۵ در سازمان لیگ برگزار شود. انتهای پیام /\n\n\n',
 'url': 'https://www.farsnews.ir/news/14001224000865/اعلام-برنامه-نشست-خبری-گل-محمدی-مجیدی-هم-باید-به-محل-تمرین-پرسپولیس'}

In [107]:
import re
from hazm import Normalizer, word_tokenize, Stemmer
from collections import Counter

ABBREVIATIONS = {
    "دک": "دانشکده",
    "ق.ظ": "قبل از ظهر",
    "خ.ا": "خدمت امام",
}

EMAIL_PATTERN = re.compile(r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)")
ID_PATTERN = re.compile(r"(@[\w_]+)")

SPACING_PATTERNS = [
    # Suffixes like ها، هاي، هایی، تر، تري، ترین، ...
    (r"(\S)(ها|هاي|هایی|تر|تري|ترین|گر|گري|ام|ات|اش)(\s|$)", r"\1‌\2 "),
    # mi or nmi => zero-width non-joiner with next token
    (r"\b(می|نمی)\s+(\S)", r"\1‌\2"),
]

DIACRITICS_PATTERN = r"[\u064b\u064c\u064d\u064e\u064f\u0650\u0651\u0652\u0670]+"
EXTRA_PUNCT_PATTERN = r"[!<>.,؛،:\-–_=+(){}\[\]…\"\'?؟«»٪%]+"
normalizer = Normalizer()


def expand_abbreviations(text):
    """Expands abbreviations using the ABBREVIATIONS dictionary."""
    for short_form, expansion in ABBREVIATIONS.items():
        pattern = fr"\b{re.escape(short_form)}\b"
        text = re.sub(pattern, expansion, text)
    return text


def preserve_emails_ids(text):
    """Wrap emails and IDs with markers to ensure they are treated as single tokens."""
    text = EMAIL_PATTERN.sub(r"EMAIL_\1_EMAIL", text)
    text = ID_PATTERN.sub(r"ID_\1_ID", text)
    return text


def spacing_correction(text):
    """Applies spacing corrections."""
    for pattern, repl in SPACING_PATTERNS:
        text = re.sub(pattern, repl, text)
    return text


def remove_diacritics_and_punct(text):
    """Removes diacritics and extra punctuation."""
    text = re.sub(DIACRITICS_PATTERN, "", text)  # Remove diacritics
    text = re.sub(EXTRA_PUNCT_PATTERN, "", text)  # Remove punctuation
    return text



def preprocess_single_text(text, expand_abbr=True, preserve_email_id=True, do_spacing_corr=True,
                           remove_diacritics=True):
    if expand_abbr:
        text = expand_abbreviations(text)

    if preserve_email_id:
        text = preserve_emails_ids(text)

    text = normalizer.normalize(text)  # Hazm normalization

    if do_spacing_corr:
        text = spacing_correction(text)

    if remove_diacritics:
        text = remove_diacritics_and_punct(text)

    tokens = word_tokenize(text)  # Tokenize using Hazm
    return tokens

In [108]:
from collections import Counter
from hazm import Normalizer, word_tokenize, Stemmer

stemmer = Stemmer()

def stem_tokens(tokens):
    """Applies stemming to a list of tokens."""
    return [stemmer.stem(t) for t in tokens]

def compute_top_k_frequent(tokens, k):
    """Computes the top-K most frequent tokens along with their frequencies."""
    token_counts = Counter(tokens)
    sorted_tokens = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)
    return dict(sorted_tokens[:k])

def simple_preprocess(content, top_k_tokens=None):
    """
    Simplified preprocessing pipeline for a single document:
      - Normalize and tokenize the content.
      - Stem tokens.
      - Optionally remove tokens present in `top_k_tokens`.
    """
    # Step 1: Normalize and tokenize the content
    tokens = preprocess_single_text(content)  # Preprocessing (normalization, tokenization)

    # Step 2: Stem tokens
    tokens = stem_tokens(tokens)  # Apply stemming

    # Step 3: Remove top-K frequent tokens if provided
    if top_k_tokens:
        tokens = [token for token in tokens if token not in top_k_tokens]

    return tokens

def preprocess_all_docs(docs, top_k=50):
    """
    Preprocesses all documents while returning the same structure as required:
      - Normalizes, tokenizes, and stems content for each document.
      - Computes top-K frequent tokens across all documents.
      - Removes top-K frequent tokens from each document's content.
    """
    combined_tokens = []
    all_tokens = {}

    # Preprocess each document
    for doc_id, doc_data in docs.items():
        content = doc_data['content']
        tokens = simple_preprocess(content)  # Use simple_preprocess to ensure stemming
        all_tokens[doc_id] = tokens
        combined_tokens.extend(tokens)

    # Compute top-K frequent tokens
    top_k_tokens_with_counts = compute_top_k_frequent(combined_tokens, top_k)
    top_k_tokens = set(top_k_tokens_with_counts.keys())  # Extract tokens only

    # Remove top-K frequent tokens from each document
    for doc_id in all_tokens:
        filtered_tokens = [token for token in all_tokens[doc_id] if token not in top_k_tokens]
        docs[doc_id]['content'] = filtered_tokens

    return docs, top_k_tokens_with_counts  # Return both processed docs and top-K tokens

In [109]:
docs, contents, _ = load_docs()

pre_processed_docs, top_k_tokens_with_counts = preprocess_all_docs(docs, top_k=20)

print("\nTop-K Frequent Tokens:")
for token, count in top_k_tokens_with_counts.items():
    print(f"Token: {token}, Count: {count}")

print("\nProcessed Document Tokens:")
for doc_id, doc_data in list(pre_processed_docs.items())[:5]:  # Display first 5 documents
    print(f"Doc {doc_id} final tokens:", doc_data['content'])


Top-K Frequent Tokens:
Token: و, Count: 234908
Token: در, Count: 165135
Token: به, Count: 136045
Token: از, Count: 92977
Token: این, Count: 83094
Token: که, Count: 75480
Token: با, Count: 69233
Token: را, Count: 68677
Token: اس, Count: 48513
Token: برا, Count: 31029
Token: کرد, Count: 26941
Token: آن, Count: 24578
Token: ه, Count: 24416
Token: یک, Count: 22864
Token: کشور, Count: 22345
Token: ت, Count: 22173
Token: ما, Count: 19785
Token: خود, Count: 18861
Token: بر, Count: 18807
Token: شد, Count: 17270

Processed Document Tokens:
Doc 0 final tokens: ['گزار', 'خبرگزار', 'فارس', 'کنفدراسیون', 'فوتبال', 'آسیا', 'AFC', 'نامه', 'رسم', 'فدراسیون', 'فوتبال', 'ایر', 'باشگاه', 'گیت', 'پسند', 'ز', 'قرعه\u200cکش', 'ج', 'باشگاه', 'فوتسال', 'آسیا', 'رسما', 'اعل', 'اساس', '۲۵', 'فروردین\u200cماه', '۱۴۰۱', 'مراس', 'قرعه\u200cکش', 'ج', 'باشگاه', 'فوتسال', 'آسیا', 'مالز', 'برگزار', 'می\u200cشود', 'باشگاه', 'گیت', 'پسند', 'بعنو', 'قهر', 'فوتسال', 'ایر', 'سال', '۱۴۰۰', 'مسابق', 'راه', 'پیدا', 'کرده_اس'

In [110]:
pre_processed_docs['10']

{'title': 'تراکتور در آستانه بازگشت به خانه اصلی/ نواقص ورزشگاه یادگار امام بررسی شد',
 'content': ['گزار',
  'خبرگزار',
  'فارس',
  'نقل',
  'سا',
  'باشگاه',
  'تراکتور',
  'میر',
  'معصو',
  'سهراب',
  'مدیرعامل',
  'باشگاه',
  'همراه',
  'نمایندگان',
  'استاندار',
  'آذربایج',
  'شرق',
  'معاون',
  'عملی',
  'نیرو',
  'انتظام',
  'نمایندگان',
  'یگ',
  'ویژه',
  'مدیرکل',
  'ورز',
  'جوان',
  'نماینده',
  'اداره',
  'ورز',
  'جوان',
  'شهرس',
  'تبریز',
  'مدیر',
  'ورزشگاه',
  'یادگار',
  'ام',
  'امروز',
  'سه\u200cشنبه',
  '۲۴',
  'اسفند',
  'بخش',
  'مختلف',
  'ورزشگاه',
  'یادگار',
  'ام',
  'ره',
  'تبریز',
  'بازدید',
  'کرده',
  'آخرین',
  'وضع',
  'نواقص',
  'مجموعه',
  'میزبان',
  'هوادار',
  'تراکتور',
  'بررس',
  'کردند',
  'ط',
  'بازدید',
  'تمهید',
  'لاز',
  'راستا',
  'آماده\u200cساز',
  'ورزشگاه',
  'میزبان',
  'بازی',
  'تراکتور',
  'اندیشیده_شد',
  'انت',
  'پی',
  '/'],
 'url': 'https://www.farsnews.ir/news/14001224000804/تراکتور-در-آستانه-بازگشت-به-خانه-اصلی-ن

In [111]:
def Postings_List(Docs, champ_len):
    my_dict = {}
    for index in Docs:
        for position, token in enumerate(Docs[index]['content']):
            # if its not new token
            if token in my_dict:
                # if the doc has already been in posting list of that token
                if index in my_dict[token]['docs']:
                    my_dict[token]['docs'][index]['positions'].append(position)
                    my_dict[token]['docs'][index]['number_of_token'] += 1 
                else:
                    my_dict[token]['docs'][index] = { 
                                                    'positions': [position],
                                                    'number_of_token': 1
                                                    }
                my_dict[token]['frequency'] += 1
                
            # add token to dictionary if its new
            else:
                my_dict[token] = {
                 'frequency': 1,
                 'docs': {
                       index: {
                           'positions': [position],
                           'number_of_token': 1
                           }
                    }
                }
                
    N = len(Docs)
    docs_vectors = {}
    
    for term in my_dict:
        term_docs = dict(my_dict[term]['docs'])
        n_t = len(term_docs)
        # calculting tf_idf for each doc in posting list
        for doc in my_dict[term]['docs']:
            tf = my_dict[term]['docs'][doc]['number_of_token']
            x = (np.log10( N / n_t ))*(1+np.log10(tf))
            my_dict[term]['docs'][doc]['tf_idf'] = x
            
            # add weight of this term to docs vector for future usages in query processing
            if doc not in docs_vectors:
                docs_vectors[doc] = {}
            docs_vectors[doc][term] = {'tf_idf':x,'tf':tf}

            
        # sort posting list and put it in champion list of each term
        sorted_term_docs = sorted(term_docs, key=lambda doc: term_docs[doc]['number_of_token'], reverse=True)
        my_dict[term]['champions_list'] = {}
        for doc_number in sorted_term_docs:
            my_dict[term]['champions_list'][doc_number] = {'number_of_token': my_dict[term]['docs'][doc_number]['number_of_token']
                                                           ,'tf_idf' : my_dict[term]['docs'][doc_number]['tf_idf']}
            
        # if champ_len is smaller than actual postings, we extract top champ_len of sorted postings 
        if champ_len < n_t:
            my_dict[term]['champions_list'] = dict(list(my_dict[term]['champions_list'].items())[:champ_len])
        
    return my_dict , docs_vectors

In [112]:
global dictionary, docs_vectors
dictionary, docs_vectors = Postings_List(pre_processed_docs, 20)

In [113]:
docs_vectors['8535']

{'گزار': {'tf_idf': 0.06822999816007738, 'tf': 1},
 'خبرگزار': {'tf_idf': 0.006888013253462664, 'tf': 1},
 'مراس': {'tf_idf': 1.3082797702727251, 'tf': 1},
 'برگزار': {'tf_idf': 0.7236736350395699, 'tf': 2},
 'می\u200cشود': {'tf_idf': 0.7264740996744733, 'tf': 3},
 'عنو': {'tf_idf': 0.49503246937512, 'tf': 1},
 'مق': {'tf_idf': 1.1250099265899205, 'tf': 1},
 'انت': {'tf_idf': 0.004435995384070603, 'tf': 1},
 'پی': {'tf_idf': 0.003000072495857743, 'tf': 1},
 '/': {'tf_idf': 0.004364086371255635, 'tf': 1},
 'ساز': {'tf_idf': 0.7613260376849612, 'tf': 1},
 'مل': {'tf_idf': 0.486329764964978, 'tf': 1},
 'سو': {'tf_idf': 0.6778518952477012, 'tf': 1},
 'زیر': {'tf_idf': 0.8060803276103631, 'tf': 1},
 'برنامه': {'tf_idf': 0.7438109781030208, 'tf': 1},
 'ورز': {'tf_idf': 1.1212293196304566, 'tf': 1},
 'امور': {'tf_idf': 1.006526552989648, 'tf': 1},
 'رقاب': {'tf_idf': 1.371263662807911, 'tf': 1},
 'نه': {'tf_idf': 0.9870957429704109, 'tf': 1},
 'میزبان': {'tf_idf': 1.4024838899048566, 'tf': 1}

In [114]:
dictionary['فارس']['champions_list']

{'7435': {'number_of_token': 64, 'tf_idf': 0.0059070807944042346},
 '11697': {'number_of_token': 36, 'tf_idf': 0.0053810823065394255},
 '163': {'number_of_token': 35, 'tf_idf': 0.005355328462095761},
 '6404': {'number_of_token': 35, 'tf_idf': 0.005355328462095761},
 '1322': {'number_of_token': 33, 'tf_idf': 0.00530153641698586},
 '1633': {'number_of_token': 31, 'tf_idf': 0.00524438024289291},
 '6755': {'number_of_token': 27, 'tf_idf': 0.00511808306260702},
 '7584': {'number_of_token': 26, 'tf_idf': 0.005083580816288167},
 '821': {'number_of_token': 25, 'tf_idf': 0.005047725200337222},
 '2388': {'number_of_token': 24, 'tf_idf': 0.0050104056913280555},
 '7744': {'number_of_token': 24, 'tf_idf': 0.0050104056913280555},
 '525': {'number_of_token': 19, 'tf_idf': 0.00479683475620343},
 '8680': {'number_of_token': 16, 'tf_idf': 0.004639729076116687},
 '10183': {'number_of_token': 16, 'tf_idf': 0.004639729076116687},
 '10025': {'number_of_token': 15, 'tf_idf': 0.00458072789429455},
 '2831': {'

In [115]:
dictionary['کمیسیون']

{'frequency': 6298,
 'docs': {'13': {'positions': [79, 91],
   'number_of_token': 2,
   'tf_idf': 1.084825715800724},
  '89': {'positions': [301],
   'number_of_token': 1,
   'tf_idf': 0.8338206800889957},
  '100': {'positions': [149],
   'number_of_token': 1,
   'tf_idf': 0.8338206800889957},
  '106': {'positions': [34],
   'number_of_token': 1,
   'tf_idf': 0.8338206800889957},
  '140': {'positions': [492],
   'number_of_token': 1,
   'tf_idf': 0.8338206800889957},
  '461': {'positions': [457],
   'number_of_token': 1,
   'tf_idf': 0.8338206800889957},
  '465': {'positions': [289],
   'number_of_token': 1,
   'tf_idf': 0.8338206800889957},
  '472': {'positions': [295],
   'number_of_token': 1,
   'tf_idf': 0.8338206800889957},
  '479': {'positions': [51],
   'number_of_token': 1,
   'tf_idf': 0.8338206800889957},
  '545': {'positions': [33, 66],
   'number_of_token': 2,
   'tf_idf': 1.084825715800724},
  '708': {'positions': [13],
   'number_of_token': 1,
   'tf_idf': 0.8338206800889

In [116]:
len(dictionary)

48168

In [117]:
import json

# Split the dictionary into two parts because of large file
half_length = len(dictionary) // 2
first_half = {key: dictionary[key] for key in list(dictionary.keys())[:half_length]}
second_half = {key: dictionary[key] for key in list(dictionary.keys())[half_length:]}

# Write the first half to the first file
with open('first_half.json', "w", encoding="utf-8") as first_file:
    json.dump(first_half, first_file, indent=4)

# Write the second half to the second file
with open('second_half.json', "w", encoding="utf-8") as second_file:
    json.dump(second_half, second_file, indent=4)

In [118]:
import math
def calculate_tf_idf(f_td, N, n_t):
    tf = 1 + np.log10(f_td)
    idf = np.log10(N / n_t)
    return tf * idf

In [119]:
def vector_length(vector_dict):
    length = math.sqrt(sum(tf_idf_value['tf_idf'] ** 2 for tf_idf_value in vector_dict.values()))
    return length

In [120]:
def query_scoring(query, total_number_of_docs, dictionary, k, champion_list = False):
    cosine_scores = {}
    jaccard_scores = {}
    query_tokens = simple_preprocess(query)
    query_tokens_count = dict(collections.Counter(query_tokens))    
    print(query_tokens_count)
    query_terms_num = sum(query_tokens_count.values())
    for term in query_tokens_count:
        if term in dictionary:
            if champion_list: 
                term_docs = dictionary[term]['champions_list']
            else:
                term_docs = dictionary[term]['docs']
            w_tq = calculate_tf_idf(query_tokens_count[term], total_number_of_docs, len(term_docs))
            for doc in term_docs:
                w_td = term_docs[doc]['tf_idf']
                #update doc scores for cosines similarity
                if int(doc) in cosine_scores:
                    #update doc scores for cosines similarity
                    cosine_scores[int(doc)] += w_td * w_tq
                    #update doc scores for jaccard similarity
                    jaccard_scores[int(doc)] += 1
                else:
                    cosine_scores[int(doc)] = w_td * w_tq
                    #update doc scores for jaccard similarity
                    jaccard_scores[int(doc)] = 1
    # calculate cosine score by dividing by doc vector length
    for doc_number in cosine_scores:
        cosine_scores[doc_number] /= vector_length(docs_vectors[str(doc_number)])
    # calculate jaccard score
    for doc_number in jaccard_scores:
        jaccard_scores[doc_number] /= (len(pre_processed_docs[str(doc_number)]['content']) + query_terms_num - jaccard_scores[doc_number])
    # sort scores for top k 
    sorted_doc_cosine = sorted(cosine_scores.items(), key=lambda x:x[1], reverse=True)
    sorted_doc_jaccard = sorted(jaccard_scores.items(), key=lambda x:x[1], reverse=True)
    return sorted_doc_cosine[:k], sorted_doc_jaccard[:k]

In [121]:
def print_results(results):
    dict_result = {}
    for rank, result in enumerate(results):
        doc_id = result[0]
        if doc_id == None:
            continue
        print(100*'-' + '\n')
        print(f'Rank: {rank + 1}')
        print(f'ID: {doc_id}')
        print(f'{docs[f"{doc_id}"]["title"]}')
        print(f'{docs[f"{doc_id}"]["url"]}')
        dict_result[rank + 1] = {'docID': doc_id,
                                 'title': docs[str(doc_id)]["title"],
                                 'url'  : docs[str(doc_id)]["url"]}
    return dict_result
        
def query_search(query, result_numbers = 5, champion_list = False):
    results1, results2 = query_scoring(query, len(docs), dictionary, result_numbers, champion_list)
    # print(results1)
    if len(results1) == 0 and len(results2) == 0:
        print("نتیجه ای یافت نشد")
    else:
        print("Cosine Scores:")
        print(100*'=')
        r1 = print_results(results1)
        print()
        print("Jaccard Scores:")
        print(100*'=')
        r2 = print_results(results2)
    return r1, r2

In [122]:
r1, r2 = query_search('کمیسیون اجتهاد', result_numbers = 5, champion_list = True)


{'کمیسیون': 1, 'اجتهاد': 1}
Cosine Scores:
----------------------------------------------------------------------------------------------------

Rank: 1
ID: 12141
«ماموستا عبدالسلام کریمی» مشاور رئیس جمهور در امور اقوام و اقلیت‌های دینی و مذهبی شد
https://www.farsnews.ir/news/14000725000846/ماموستا-عبدالسلام-کریمی-مشاور-رئیس-جمهور-در-امور-اقوام-و-اقلیت‌های
----------------------------------------------------------------------------------------------------

Rank: 2
ID: 11022
رستمی: چاره‌ای غیر از حرکت به سمت تحول در علوم انسانی نداریم
https://www.farsnews.ir/news/14000824000581/رستمی-چاره‌ای-غیر-از-حرکت-به-سمت-تحول-در-علوم-انسانی-نداریم
----------------------------------------------------------------------------------------------------

Rank: 3
ID: 11446
دستورکار کمیسیون‌های مجلس/ بررسی تفحص از شستا و اتاق بازرگانی و دیدار با اژه‌ای و مخبر
https://www.farsnews.ir/news/14000815000142/دستورکار-کمیسیون‌های-مجلس-بررسی-تفحص-از-شستا-و-اتاق-بازرگانی-و-دیدار
------------------------------------